In [144]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine 
import pandas as pd
import numpy as np

In [145]:
import sqlalchemy
sqlalchemy.__version__

'1.4.46'

In [146]:
# Function to extract table to a pandas DataFrame
def extract_table_to_pandas(tablename, db_engine):
    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine)

db_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('subnet_user', 'subnet_user123', 
                                                          'localhost', 'elearning'))

In [147]:
def print_user_comparison(user1, user2, user3):
    print("Course id overlap between users:")
    print("================================")
    print("User 1 and User 2 overlap: {}".format(
    set(user1.course_id) & set(user2.course_id)
    ))
    print("User 1 and User 3 overlap: {}".format(
    set(user1.course_id) & set(user3.course_id)
    ))
    print("User 2 and User 3 overlap: {}".format(
    set(user2.course_id) & set(user3.course_id)
    ))

In [148]:
# Get user with id 4387
user1 = pd.read_sql("SELECT * FROM rating where user_id=4387", db_engine)

# Get user with id 18163
user2 = pd.read_sql("SELECT * FROM rating where user_id=18163", db_engine)

# Get user with id 8770
user3 = pd.read_sql("SELECT * FROM rating where user_id=8770", db_engine)

# Use the helper function to compare the 3 users
print_user_comparison(user1, user2, user3)

Course id overlap between users:
User 1 and User 2 overlap: {32, 96, 36, 6, 7, 44, 95}
User 1 and User 3 overlap: set()
User 2 and User 3 overlap: set()


In [149]:
def extract_rating_data(db_engine):
    return pd.read_sql("SELECT * FROM rating", db_engine)

def extract_course_data(db_engine):
    return pd.read_sql("SELECT * FROM courses", db_engine)

In [150]:
ratings = extract_rating_data(db_engine)

course_data = extract_course_data(db_engine)

In [151]:
ratings.head()

,user_id,course_id,rating
0,1,6,4
1,1,36,5
2,1,37,5
3,1,45,5
4,1,50,5


In [152]:
course_data.head()

,course_id,title,description,programming_language
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets
2,3,Intermediate R,The intermediate R course is the logical next ...,r
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r


In [153]:
ratings.user_id.nunique()

38174

In [154]:
course_data.shape

(100, 4)

In [155]:
# Print out the number of missing values per column
print(course_data.isnull().sum())

course_id               0
title                   0
description             0
programming_language    3
dtype: int64


In [156]:
# The transformation should fill in the missing values
def transform_fill_programming_language(course_data):
    imputed = course_data.fillna({"programming_language": "R"})
    return imputed

transformed = transform_fill_programming_language(course_data)

# Print out the number of missing values per column of transformed
print(transformed.isnull().sum())

course_id               0
title                   0
description             0
programming_language    0
dtype: int64


In [157]:
# transformation function
def transform_avg_rating(rating_data):
    # Group by course_id and extract average rating per course
    avg_rating = rating_data.groupby('course_id').rating.mean()
    # Return sorted average ratings per course
    sort_rating = avg_rating.sort_values(ascending=False).reset_index()
    return sort_rating

In [158]:
# Use transform_avg_rating on the extracted data and print results
avg_course_ratings = transform_avg_rating(ratings)
print(avg_course_ratings) 

    course_id    rating
0          46  4.800000
1          23  4.800000
2          96  4.692765
3          56  4.661765
4          24  4.653061
..        ...       ...
94         54  4.238095
95         92  4.222222
96         29  4.208333
97         17  4.147059
98         42  4.107570

[99 rows x 2 columns]


In [159]:
def get_courses_to_recommend(ratings, course_data):
    user_list = ratings.user_id.unique()

    course_set = set(course_data.course_id.unique())

    non_enrolled_users = []

    for user in user_list:
        rated_course_set = set(ratings[ratings.user_id == user].course_id.unique())
        non_enrolled_courses = course_set - rated_course_set
        non_enrolled_users = non_enrolled_users + [(user, cid) for cid in non_enrolled_courses]
        if(user == 50):
            break
    non_enrolled_courses_df = pd.DataFrame(non_enrolled_users, columns = ['user_id', 'course_id'])
    return non_enrolled_courses_df

courses_to_recommend = get_courses_to_recommend(ratings, course_data)
courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
# courses_to_recommend.sample(n=50).to_csv('courses_to_recommend_small.csv', index = False)

In [160]:
course_data.shape

(100, 4)

In [161]:
temp = pd.read_csv('courses_to_recommend.csv')
temp.sample(n=900).to_csv('courses_to_recommend_small.csv', index = False)

In [162]:
temp.shape

(4851, 2)

In [163]:
courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')

courses_to_recommend.head()

,user_id,course_id
0,49,57
1,37,56
2,42,100
3,20,47
4,27,75


In [164]:
courses_to_recommend['course_duration'] = np.random.randint(2, 11, size=len(courses_to_recommend))

In [165]:
courses_to_recommend.groupby('user_id').count()

,course_id,course_duration
user_id,,
1,19,19
2,14,14
3,15,15
4,25,25
5,17,17
6,16,16
7,22,22
8,20,20
9,20,20


In [166]:
# Complete the transformation function
def transform_recommendations(avg_course_ratings, courses_to_recommend):
    # Merge both DataFrames
    merged = courses_to_recommend.merge(avg_course_ratings) 
    
    # Sort values by rating and group by user_id
    grouped = merged.sort_values("rating", ascending=False).groupby("user_id")
    
    # Produce the top 3 values and sort by user_id
    recommendations = grouped.head(5).sort_values("user_id").reset_index()
    
    final_recommendations = recommendations[["user_id", "course_id","rating","course_duration"]]
    
    # Return final recommendations
    return final_recommendations


# Use the function with the predefined DataFrame objects
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [167]:
recommendations.head()

,user_id,course_id,rating,course_duration
0,1,31,4.610209,2
1,1,38,4.542857,6
2,1,24,4.653061,10
3,1,85,4.627119,2
4,1,86,4.608696,6


In [168]:
# Merge both DataFrames
merged = courses_to_recommend.merge(avg_course_ratings) 

merged.head()

,user_id,course_id,course_duration,rating
0,49,57,10,4.411765
1,9,57,2,4.411765
2,30,57,6,4.411765
3,41,57,7,4.411765
4,8,57,7,4.411765


In [169]:
merged.sort_values("rating", ascending=False).head()

,user_id,course_id,course_duration,rating
854,30,23,5,4.8
423,48,46,8,4.8
860,25,23,10,4.8
859,45,23,10,4.8
858,35,23,2,4.8


In [172]:
merged[merged.user_id == 47]

,user_id,course_id,course_duration,rating
19,47,100,2,4.282609
128,47,70,7,4.428144
218,47,62,2,4.283019
254,47,48,10,4.425926
399,47,24,2,4.653061
414,47,93,4,4.414634
441,47,63,3,4.523810
539,47,11,3,4.486034
579,47,79,3,4.395833
600,47,14,5,4.593220


In [173]:
# Sort values by rating and group by user_id
grouped = merged.sort_values("rating", ascending=False).groupby("user_id")


# Produce the top 3 values and sort by user_id
recommendations = grouped.head(3).sort_values("user_id")

recommendations.head()

,user_id,course_id,course_duration,rating
397,1,24,10,4.653061
763,1,85,2,4.627119
491,1,31,2,4.610209
641,2,35,5,4.588235
291,2,44,8,4.593866


In [174]:
recommendations[recommendations.user_id == 9]

,user_id,course_id,course_duration,rating
646,9,35,9,4.588235
420,9,46,2,4.800000
10,9,56,8,4.661765


In [175]:
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [176]:
recommendations.head(10)

,user_id,course_id,rating,course_duration
0,1,31,4.610209,2
1,1,38,4.542857,6
2,1,24,4.653061,10
3,1,85,4.627119,2
4,1,86,4.608696,6
5,2,44,4.593866,8
6,2,3,4.572611,8
7,2,83,4.545455,9
8,2,35,4.588235,5
9,2,74,4.529412,8


In [177]:


def load_to_dwh(recommendations, db_engine):
    recommendations.to_sql("recommendations", db_engine, if_exists="replace")

In [178]:
load_to_dwh(recommendations, db_engine)

In [179]:
def recommendations_for_user(user_id, threshold , programming_language, course_duration):
    # Join with the courses table
    query = """
    SELECT title, rating FROM recommendations 
    INNER JOIN courses ON courses.course_id = recommendations.course_id
    WHERE user_id=%(user_id)s AND rating>%(threshold)s AND programming_language = %(programming_language)s AND course_duration<%(course_duration)s;
    """
    # Add the threshold parameter
    predictions_df = pd.read_sql(query, db_engine, params = {"user_id": user_id, 
                                                           "threshold": threshold, 
                                                            "programming_language":programming_language,
                                                            'course_duration':course_duration
                                                            })
    return predictions_df.title.values

# Try the function you created

print(recommendations_for_user(4, 3, "r",8))

['Introduction to R for Finance'
 'Exploratory Data Analysis in R: Case Study']


## ETL: Combine all the operations

In [128]:
def etl(db_engine):
    # Extract the data
    courses = extract_course_data(db_engine)
    rating = extract_rating_data(db_engine)
    
    # Clean up courses data
    courses = transform_fill_programming_language(courses)
    
    # Get the average course ratings
    avg_course_rating = transform_avg_rating(rating)
    
    # Get eligible user and course id pairs
    courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')
    
    # Calculate the recommendations
    recommendations = transform_recommendations(
    avg_course_rating,
    courses_to_recommend,
    )
    
    # Load the recommendations into the database
    load_to_dwh(recommendations, db_engine)

In [130]:
etl(db_engine)

# Main function

In [180]:
a = int(input('Enter user id:'))
b = float(input('Enter minimum rating of course:'))
c = str(input('Enter language:'))
d = int(input('Enter the maximum length of preferred language course(in hrs) : '))

print(recommendations_for_user(a,b,c,d)) 

Enter user id:23
Enter minimum rating of course:3.6
Enter language:r
Enter the maximum length of preferred language course(in hrs) : 9
['Importing Data in R (Part 1)']


In [181]:
a = int(input('Enter user id:'))
b = float(input('Enter minimum rating of course:'))
c = str(input('Enter language:'))
d = int(input('Enter the maximum length of preferred language course(in hrs) : '))

print(recommendations_for_user(a,b,c,d)) 

Enter user id:49
Enter minimum rating of course:4
Enter language:python
Enter the maximum length of preferred language course(in hrs) : 5
['Building Recommendation Engines with PySpark']


In [194]:
print(recommendations_for_user(47, 2, "python",10))

['Intro to Portfolio Risk Management in Python']
